In [1]:
import cv2
import itertools
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getSize(image, face_landmarks, INDEXES):
    '''
    This function calculates the height and width of a face part utilizing its landmarks.
    Args:
        image:          The image of person(s) whose face part size is to be calculated.
        face_landmarks: The detected face landmarks of the person whose face part size is to 
                        be calculated.
        INDEXES:        The indexes of the face part landmarks, whose size is to be calculated.
    Returns:
        width:     The calculated width of the face part of the face whose landmarks were passed.
        height:    The calculated height of the face part of the face whose landmarks were passed.
        landmarks: An array of landmarks of the face part whose size is calculated.
    '''
    
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape
    
    # Initialize a list to store the landmarks of the face part.
    landmarks = []
    
    # Iterate over the indexes of the landmarks of the face part. 
    for INDEX in INDEXES:
        
        # Append the landmark into the list.
        landmarks.append([int(face_landmarks.landmark[INDEX].x * image_width),
                               int(face_landmarks.landmark[INDEX].y * image_height)])
    
    # Calculate the width and height of the face part.
    # TODO: Use the 3D box instead of 2D rectangle
    _, _, width, height = cv2.boundingRect(np.array(landmarks))
    
    # Convert the list of landmarks of the face part into a numpy array.
    landmarks = np.array(landmarks)
    
    # Return the calculated width height and the landmarks of the face part.
    return width, height, landmarks

In [3]:
def isOpen(image, face_mesh_results, face_part, threshold=5):
    '''
    This function checks whether the eye or mouth of the person(s) is open, 
    utilizing its facial landmarks.
    Args:
        image:             The image of person(s) whose an eye or mouth is to be checked.
        face_mesh_results: The output of the facial landmarks detection on the image.
        face_part:         The name of the face part that is required to check: MOUTH, RIGHT EYE, LEFT EYE
        threshold:         The threshold value used to check the isOpen condition.
    Returns:
        status:       A dictionary containing isOpen statuses of the face part of all the 
                      detected faces.  
    '''
    
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape

    
    # Create a dictionary to store the isOpen status of the face part of all the detected faces.
    status={}
    
    # Check if the face part is mouth.
    if face_part == 'MOUTH':
        
        # Get the indexes of the mouth.
        INDEXES = mp_face_mesh.FACEMESH_LIPS
        
    # Check if the face part is left eye.    
    elif face_part == 'LEFT EYE':
        
        # Get the indexes of the left eye.
        INDEXES = mp_face_mesh.FACEMESH_LEFT_EYE
    
    # Check if the face part is right eye.    
    elif face_part == 'RIGHT EYE':
        
        # Get the indexes of the right eye.
        INDEXES = mp_face_mesh.FACEMESH_RIGHT_EYE 
           
    # Otherwise return nothing.
    else:
        return
    
    # Convert the indexes of the landmarks of the face part into a list of unique indices.
    INDEXES_LIST = set(list(itertools.chain(*INDEXES)))
    
    # Iterate over the found faces.
    for face_no, face_landmarks in enumerate(face_mesh_results.multi_face_landmarks):
        
        # Get the height of the face part.
        _, height, _ = getSize(image, face_landmarks, INDEXES_LIST)
        
        # Get the height of the whole face.
        face_oval = set(list(itertools.chain(*mp_face_mesh.FACEMESH_FACE_OVAL)))
        _, face_height, _ = getSize(image, face_landmarks, face_oval)
        
        # Check if the face part is open.
        if (height/face_height)*100 > threshold:
            
            # Set status of the face part to open.
            status[face_no] = 'OPEN'
        
        # Otherwise.
        else:
            # Set status of the face part to close.
            status[face_no] = 'CLOSE'
    
    # Otherwise
    else:
        
        # Return the output image and the isOpen statuses of the face part of each detected face.
        return status

In [4]:
def checkRotation(face_landmarks):
    '''
    This function returns the order of the forward rectangle points of the 3D bounding box of the face
    Args:
        face_landmarks: Landmarks of the face
    Returns:
        -:      The number represents the order of the forward rectangle points, which are the points 0, 1, 2 and 7 
                of the array with the 8 points, in the oriented space
                0: 0*****2    1: 7*****1    2: 2*****0    3: 1*****7    4: error
                   *     *       *     *       *     *       *     *
                   *     *       *     *       *     *       *     *
                   1*****7       2*****0       7*****1       0*****2
        R: Rotation matrix of the 3D bounding box
        points: points of the 3D bounding box
    '''
    
    # Retrieve the landmarks of the face as a list
    face_landmarks_l = []
    for index in range(len(face_landmarks.landmark)):
        face_landmarks_l.append([face_landmarks.landmark[index].x, 
                          face_landmarks.landmark[index].y, 
                          face_landmarks.landmark[index].z])
    
    # Get the 3D oriented bounding box
    o3d_landmarks_f = o3d.utility.Vector3dVector(np.array(face_landmarks_l))
    o3d_bbox_f = o3d.geometry.OrientedBoundingBox.create_from_points(o3d_landmarks_f)
    
    # Get the rotation matrix from the oriented space to the aligned space
    R = o3d_bbox_f.R
    points = np.asarray(o3d_bbox_f.get_box_points())
    
    # In the aligned space the points are stored like this
    #  2*****7
    #  *     *
    #  *     *
    #  0*****1
    # So we check how they turned from oriented space to aligned space to get the rotation/inversion
    if points[2, 0] > points[1, 0] and points[2, 1] > points[1, 1]:
        return 0, R, points
    elif points[2, 0] < points[1, 0] and points[2, 1] < points[1, 1]:
        return 1, R, points
    elif points[2, 0] < points[1, 0] and points[2, 1] > points[1, 1]:
        return 2, R, points
    elif points[2, 0] > points[1, 0] and points[2, 1] < points[1, 1]:
        return 3, R, points
    else:
        return 4, R, points

In [5]:
def boundingBox(face_landmarks, face_part, INDEXES, angle, R, required_size):
    '''
    This function returns the new corner points of the filter image in the image space
    Args:
        face_landmarks: Landmarks of the face
        face_part:      Name of the face part we want to apply the filter on
        INDEXES:        Indices of the face part landmarks
        angle:          Orientation of the 3D bouding box of the entire face
        R:              The rotation matrix of the 3D bounding box
        required_size:  Required width and height of the filter in the aligned space = before applying persective
    Returns:
        forward_points: New corner points of the filter image in the oriented space
    '''
    
    # Retrieve the landmarks of the face part
    landmarks = []
    for index in INDEXES:
        landmarks.append([face_landmarks.landmark[index].x, 
                          face_landmarks.landmark[index].y, 
                          face_landmarks.landmark[index].z])
    
    # Create the 3D oriented bounding box
    o3d_landmarks = o3d.utility.Vector3dVector(np.array(landmarks))
    o3d_bbox = o3d.geometry.OrientedBoundingBox.create_from_points(o3d_landmarks)
    
    # Get box points and center point
    box_points = np.asarray(o3d_bbox.get_box_points())
    box_center = np.asarray(o3d_bbox.get_center())

    # Retrieve the inverse rotation matrix of the oriented bounding box --> R_inv put a point in the oriented space
    # to the aligned space
    R_inv = np.linalg.inv(R)
    
    # Retrieve one of the forward points to get the depth in the aligned space
    index = np.argmin(box_points[:,2])
    forward_point = box_points[index, :] 
    
    # Project the point in the aligned space and center
    forward_point_projected = np.matmul(R_inv, forward_point - box_center)
    
    # Get the required width and height
    required_width, required_height = required_size
    

    # Get the points position in aligned space in the right order
    # It depends on the filter type and orientation of the bounding box
    # We are in the aligned space and we will then put the image in the oriented space and then in the image space.
    # We want the points such that we have 0*****3 in image space, the number being the order in which they are arranged
    #                                      *     * 
    #                                      *     *
    #                                      1*****2
    # If angle == 0
    # In aligned space we have 2*****7, in oriented space 0*****2, and in image space 0*****1 for the bounding box points
    #                          *     *                    *     *                     *     *
    #                          *     *                    *     *                     *     *
    #                          0*****1                    1*****7                     2*****7
    #
    # So the the points should be written 2*****3
    #                                     *     *
    #                                     *     *
    #                                     1*****0
    # That way we will have 
    # In aligned space 2*****3, in oriented space 1*****2, and in image space 0*****3
    #                  *     *                    *     *                     *     *
    #                  *     *                    *     *                     *     *
    #                  1*****0                    0*****3                     1*****2
    forward_points_transformed = []
    if face_part == 'FOREHEAD':        
        if angle == 0:            
            forward_points_transformed.append([required_height, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height, required_width / 2, forward_point_projected[2]])
        elif angle == 1:
            forward_points_transformed.append([-required_height, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height, - required_width / 2, forward_point_projected[2]])
        elif angle == 2:
            forward_points_transformed.append([required_height, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height, - required_width / 2, forward_point_projected[2]])
        elif angle == 3:
            forward_points_transformed.append([-required_height, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height, required_width / 2, forward_point_projected[2]])
        else:
            return

            
    elif face_part == 'RIGHT EYE' or face_part == 'LEFT EYE':  
        if angle == 0:            
            forward_points_transformed.append([required_height/2, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, required_width / 2, forward_point_projected[2]])
        elif angle == 1:
            forward_points_transformed.append([-required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, - required_width / 2, forward_point_projected[2]])
        elif angle == 2:
            forward_points_transformed.append([required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, - required_width / 2, forward_point_projected[2]])
        elif angle == 3:
            forward_points_transformed.append([-required_height/2, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, required_width / 2, forward_point_projected[2]])
        else:
            return
        
    elif face_part == 'MOUTH':
        if angle == 0:            
            forward_points_transformed.append([0, -required_width / 2, 0])
            forward_points_transformed.append([-required_height, -required_width / 2, 0])
            forward_points_transformed.append([-required_height, required_width / 2, 0])
            forward_points_transformed.append([0, required_width / 2, 0])
        elif angle == 1:
            forward_points_transformed.append([0, required_width / 2, 0])
            forward_points_transformed.append([required_height, required_width / 2, 0])
            forward_points_transformed.append([required_height, - required_width / 2, 0])
            forward_points_transformed.append([0, - required_width / 2, 0])
        elif angle == 2:
            forward_points_transformed.append([0, required_width / 2, 0])
            forward_points_transformed.append([-required_height, required_width / 2, 0])
            forward_points_transformed.append([-required_height, - required_width / 2, 0])
            forward_points_transformed.append([0, - required_width / 2, 0])
        elif angle == 3:
            forward_points_transformed.append([0, -required_width / 2, 0])
            forward_points_transformed.append([required_height, -required_width / 2, 0])
            forward_points_transformed.append([required_height, required_width / 2, 0])
            forward_points_transformed.append([0, required_width / 2, 0])
        else:
            return     
    else:
        return
       
    # Get the filter image in the oriented space 
    forward_points = []
    for p in forward_points_transformed:
        p_rotated = np.matmul(R, p) # Rotate
        p_translated = p_rotated + box_center # Translate to box position
        forward_points.append(p_translated)
            
    return forward_points


In [6]:
def drawFilter(image, filter_img, face_landmarks, face_part, INDEXES, angle, R, required_size):
    '''
    This function returns camera image with the filter image added at the right place
    Args:
        image:           Camera image
        filter_image:    Filter image
        face_landmarks:  Landmarks of the entire face
        face_part:       The name of the part you want to put the filter on
        INDEXES:         Indices of the face part landmarks
        angle:           Orientation of the 3D bouding box of the entire face
        R:               The rotation matrix of the 3D bounding box
        required_size:   Required width and height of the filter in the aligned space = before applying persective
    Returns:
        annotated_image: The image with the filter added
    '''
    
    # Get width and height of the filter image
    filter_img_height, filter_img_width, _  = filter_img.shape
    
    # Get width and height of the camera image
    image_height, image_width, _ = image.shape
    
    # Get required width and height
    required_width = required_size[0]
    required_height = required_size[1]
    
    # Normalize the required width and height for aligned/oriented spaces
    required_height_norm = required_height / image_height
    required_width_norm = required_width / image_width
    
    # Get the new corners points of the filter image (in the oriented space)
    forward_points = boundingBox(face_landmarks, face_part, INDEXES, angle, R,
                                                             [required_width_norm, required_height_norm])
          
    # Get the initial corner points of the filter image (counterclockwise) in image space
    # 0*****3
    # *     *
    # *     *
    # 1*****2
    pts1 = np.array([[0, 0], [0, filter_img_height], 
                     [filter_img_width, filter_img_height], [filter_img_width, 0]])
    
    # Get the final corner points of the filter image in the overlay image (counterclockwise) in image space
    # with perspective
    pts2 = []
    for point in forward_points:
        resolution_x = 1
        resolution_y = 1

        # Add perspective
        fov = 45
        z0 = (resolution_x / 2) / np.tan((fov/2) * np.pi / 180)

        relative_x = int(((point[0] - 0.5) * z0 / (z0 + point[2]) + 0.5) * image_width)
        relative_y = int(((point[1] - 0.5) * z0 / (z0 + point[2]) + 0.5) * image_height)

        pts2.append([relative_x, relative_y])
    pts2 = np.array(pts2)

    # Find homography from the filter image to the overlay image
    h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)

    # Put the filter image in perspective
    im1Reg = cv2.warpPerspective(filter_img, h, (image_width, image_height))

    # Inverse filter image mask in perspective
    _, filter_img_mask = cv2.threshold(cv2.cvtColor(im1Reg, cv2.COLOR_BGR2GRAY),
                                   25, 255, cv2.THRESH_BINARY_INV)

    # Spread in 3 dimensions (rgb)
    filter_img_mask = np.expand_dims(filter_img_mask, axis=2)
    filter_img_mask = np.repeat(filter_img_mask, 3, axis=2)

    # Final image with black pixels at filter image position
    resultant_image = cv2.bitwise_and(image, filter_img_mask)

    # Use Bitwise or to merge the two images
    annotated_image = cv2.bitwise_or(im1Reg, resultant_image)        

    return annotated_image

In [7]:
# Initialize the mediapipe face mesh class.
mp_face_mesh = mp.solutions.face_mesh

# Setup the face landmarks function for videos.
# TODO: Here works for only one face --> Make it work for multiple faces (see overlay call in main function)
face_mesh_videos = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, 
                                         min_detection_confidence=0.5,min_tracking_confidence=0.3)

In [8]:
# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

# Initialize the mediapipe drawing styles class.
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)

# Set camera resolution
camera_video.set(3,1280)
camera_video.set(4,960)

# Create named window for resizing purposes.
cv2.namedWindow('Face Filter', cv2.WINDOW_NORMAL)

# Read the left and right eyes images.
# TODO: Create an interactive window to let the user select the wanted filters
left_eye = cv2.imread('data/left_eye_cupcake.png')
right_eye = cv2.imread('data/right_eye_cupcake.png')

crown = cv2.imread('data/crown.png')

# Initialize the VideoCapture object to read from the smoke animation video stored in the disk.
# TODO: Same as for images --> Create a dictionary to store all images and videos?
animation = cv2.VideoCapture('data/rainbow_animation1.mp4')

# Set the smoke animation video frame counter to zero.
animation_frame_counter = 0

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then continue to the next iteration to read
    # the next frame.
    if not ok:
        continue
        
    # Get the width and height of the image
    image_height, image_width, _ = frame.shape
        
    # Read a frame from smoke animation video
    _, animation_frame = animation.read()
    
    # Increment the smoke animation video frame counter.
    animation_frame_counter += 1
    
    # Check if the current frame is the last frame of the animation video.
    if animation_frame_counter == animation.get(cv2.CAP_PROP_FRAME_COUNT):     
        
        # Set the current frame position to first frame to restart the video.
        animation.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
        # Set the animation video frame counter to zero.
        animation_frame_counter = 0
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Perform Face landmarks detection.
    face_mesh_results = face_mesh_videos.process(frame[:,:,::-1])    
    
    # Check if facial landmarks are found.
    if face_mesh_results.multi_face_landmarks:        
        # Get the mouth isOpen status of the person in the frame.
        mouth_status = isOpen(frame, face_mesh_results, 'MOUTH', 
                                     threshold=15)
        
        # Get the left eye isOpen status of the person in the frame.
        left_eye_status = isOpen(frame, face_mesh_results, 'LEFT EYE', 
                                        threshold=4.5)
        
        # Get the right eye isOpen status of the person in the frame.
        right_eye_status = isOpen(frame, face_mesh_results, 'RIGHT EYE', 
                                         threshold=4.5)
        
        # Iterate over the found faces.
        for face_num, face_landmarks in enumerate(face_mesh_results.multi_face_landmarks):
            
            # Check the orientation of the face bounding box
            angle, R, points = checkRotation(face_landmarks)
            
            # Check if the left eye of the face is open.
            if left_eye_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = left_eye.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                left_eye_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
                
                # Get the height of the face part on which we will overlay the filter image.
                _, face_part_height, landmarks = getSize(frame, face_landmarks, left_eye_landmarks)

                # Specify the height to which the filter image is required to be resized.
                # 2.5 can be changed depending on the size of the filter we want
                # This allows the filter to be bigger/smaller depending on both the size of the eye 
                # and the size of the aperture of the eye
                required_height = int(face_part_height*2.5)
                required_width = int(filter_img_width*(required_height/filter_img_height))
                
                # Draw the filter
                frame = drawFilter(frame, left_eye, face_landmarks, 'LEFT EYE',
                                   left_eye_landmarks, angle, R, [required_width, required_height])
                
            
            # Check if the right eye of the face is open.
            if right_eye_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = right_eye.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                right_eye_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))
                
                # Get the height of the face part on which we will overlay the filter image.
                _, face_part_height, landmarks = getSize(frame, face_landmarks, right_eye_landmarks)

                # Specify the height to which the filter image is required to be resized.
                # 2.5 can be changed depending on the size of the filter we want
                # This allows the filter to be bigger/smaller depending on both the size of the eye 
                # and the size of the aperture of the eye
                required_height = int(face_part_height*2.5)
                required_width = int(filter_img_width*(required_height/filter_img_height))
                
                # Draw the filter
                frame = drawFilter(frame, right_eye, face_landmarks, 'RIGHT EYE',
                                   right_eye_landmarks, angle, R, [required_width, required_height])
                            
            # Check if the mouth of the face is open.
            if mouth_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = animation_frame.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                lips_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))
                
                # Get the height of the face part on which we will overlay the filter image.
                face_part_width, face_part_height, landmarks = getSize(frame, face_landmarks, lips_landmarks)
                
                # Specify the height to which the filter image is required to be resized.
                # 0.7 can be changed depending on the size of the filter we want
                required_width = int(face_part_width*0.7)
                required_height = int(filter_img_height*(required_width/filter_img_width))
                
                # Draw the filter
                frame = drawFilter(frame, animation_frame, face_landmarks, 'MOUTH', 
                                   lips_landmarks, angle, R, [required_width, required_height])
            
            # FOREHEAD
            # Get the width and height of filter image.
            filter_img_height, filter_img_width, _  = crown.shape
            
            # Get the landmarks of the forehead
            forehead_landmarks = [103, 67, 109, 10, 338, 297, 332]
            
            # Get the height of the face part on which we will overlay the filter image.
            face_part_width,_, landmarks = getSize(frame, face_landmarks, forehead_landmarks)
                
            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            required_width = int(face_part_width*2.5)
            required_height = int(filter_img_height*(required_width/filter_img_width))
            
            # Draw the filter
            frame = drawFilter(frame, crown, face_landmarks, 'FOREHEAD',
                                   forehead_landmarks, angle, R, [required_width, required_height])
            

        
    
    # Display the frame.
    cv2.imshow('Face Filter', frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF    
    
    # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break

# Release the VideoCapture Object and close the windows.                  
camera_video.release()
cv2.destroyAllWindows()